In [ ]:
#Setup
!pip install tika
!pip install pandas
import pandas as pd
import tika
tika.initVM()
from tika import parser
import glob
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
my_pdf = glob.glob("/content/drive/My Drive/d4g_database/all_pdf/*.pdf")
print(my_pdf)

[]


In [ ]:
def pdf_meta():
  pdfs = glob.glob("/content/drive/My Drive/d4g_database/all_pdf/*.pdf")
  meta_list = []
  content_list = []

  for item in pdfs:
    parsed = parser.from_file(item)
    meta_list.append(parsed["metadata"])
    content_list.append(parsed['content'])
  return meta_list, content_list

In [ ]:
metadata, contentdata = pdf_meta()

In [ ]:
def df_pdfs(meta, keywords):
  unique_id = []
  title_list = []
  date_list = []
  content_list = keywords
  
  [title_list.append(d['title']) if 'title' in d else title_list.append('null') for d in meta]
  [date_list.append(d['date']) if 'date' in d else date_list.append('null') for d in meta]
  counter = 0
  for i in meta:
    unique_id.append(counter)
    counter += 1
  
  df = pd.DataFrame()
  df['ID'] = unique_id
  df['Title'] = title_list
  df['Date'] = date_list
  df['Content'] = content_list



  return df


In [ ]:
pdf_df = df_pdfs(metadata, contentdata)

In [ ]:
pdf_df

,ID,Title,Date,Content


In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('TestDB1.db')
c = conn.cursor()

c.execute('CREATE TABLE PDFS (ID text, Title text, Date text, Content text)')
conn.commit()

In [ ]:
pdf_df.to_sql('PDFS', conn, if_exists='replace', index = False)